# twoaxistracking

In [1]:
from utils import get_rtd_analytics_data, get_github_stars
import pandas as pd
import numpy as np
import datetime

from bokeh.plotting import figure, show
from bokeh.models import HoverTool
from bokeh.transform import jitter
from bokeh.io import output_notebook

In [2]:
output_notebook()

Loading BokehJS ...

## ReadTheDocs

In [3]:
df = get_rtd_analytics_data('twoaxistracking')
data_period = df['Date'].min().strftime('%Y-%m-%d') + ' to ' + df['Date'].max().strftime('%Y-%m-%d')

In [4]:
total_by_version = df.groupby('Version')['Views'].sum().reset_index()

p = figure(x_range=total_by_version['Version'], height=350, tooltips=[("Version", "@Version"), ("Views", "@Views")],
           title=f"Page views by RTD version ({data_period})",)
p.vbar(x='Version', top='Views', width=0.75, source=total_by_version,
       line_color='white')
p.xaxis.major_label_orientation = 3.14/2
p.yaxis.axis_label = 'Total page views'
show(p)

In [5]:
daily_views = df.pivot_table(values='Views', index='Date', columns='Version', aggfunc='sum')[['stable', 'latest']].fillna(0)

p = figure(x_axis_type="datetime", height=350, title=f"Daily views by RTD version")
hover_tool = HoverTool(tooltips=[('Date', '@x{%Y-%m-%d}'), ('Views', '@y')],
                       formatters={'@x': 'datetime'})
hover_tool.point_policy = 'snap_to_data'
p.add_tools(hover_tool)
p.line(daily_views.index, daily_views['stable'], legend_label='stable')
p.line(daily_views.index, daily_views['latest'], legend_label='latest', color='#ff7f0e')
p.yaxis.axis_label = 'Daily page views'
show(p)

In [6]:
df2 = df.loc[df['Version'] == 'stable', :].copy()
pathviews = df2.groupby('Path')['Views'].sum().reset_index()

In [7]:
subset = pathviews.sort_values('Views', ascending=False)
p = figure(y_range=subset['Path'], height=400, tooltips=[("URL", "@Path"), ("Views", "@Views")],
           title=f"Views by URL ({data_period})")
p.hbar(y='Path', right='Views', source=subset, height=0.75,
       line_color='white')
p.xaxis.axis_label = 'Total page views'
show(p)

## GitHub

In [8]:
gh = get_github_stars('pvlib/twoaxistracking')

In [9]:
star_curve = gh.set_index('star_date').assign(x=1).x.cumsum().resample('d').max()
# project out to present:
star_curve = pd.concat([star_curve, pd.Series({datetime.datetime.utcnow(): np.nan})])
star_curve = star_curve.ffill()

p = figure(height=350, x_axis_type="datetime")
hover_tool = HoverTool(tooltips=[('Date', '@x{%Y-%m-%d}'), ('Total Stars', '@y')],
                       formatters={'@x': 'datetime'})
hover_tool.point_policy = 'snap_to_data'
p.add_tools(hover_tool)

p.line(star_curve.index, star_curve)
p.yaxis.axis_label = 'Total Stars'
p.xaxis.axis_label = 'Date'
show(p)